In [1]:
import os
import numpy as np
import pandas as pd

import tensorflow as tf
print(tf.__version__)

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
2.5.0-rc2


### Basic architecture of a CNN

| Hyperparameter/Layer type | Description |
|:-------------------------:|-------------|
|   Input image(s)          | Images for discovering pattern |
|   Input layer             | Takes in target image and preprocess them for further layers     |
|   Convolution layer       | Extracts/Learns most importand patterns from images    |
|   Hidden activation       | Adds non-linearity to learned features       |
|   Pooling layer           | Reduces the dimensionality of learned image features        |
|   Fully connected layer   | Further refines learned features from convolution layers      |
|   Output layer            | Takes learned features and outputs them in shape of target labels       |
|   Output activation       | Adds non-linearity to output layer      |



_It's typically a stack of convolution layer and pooling layers_


For this exercise we are using dataset available at [pizza_steak](https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip)

In [26]:
# Inspect the data

import pathlib

data_dir = pathlib.Path('pizza_steak/train/')
class_names = np.array(sorted([f.name for f in data_dir.glob('*')]))

class_names

array(['pizza', 'steak'], dtype='<U5')

In [43]:
import matplotlib.image as mpimg
import random

def view_random_image(target_dir, target_class):
    
    target_folder = os.path.join(target_dir, target_class)
    
#     Get random image path
    image_path = random.sample(os.listdir(target_dir), 1)[0]
    
#     Read in image and plot it using matplotlib
    
    for i in range(4):
        img = mpimg.imread(os.path.join(target_dir, image_path))
        plt.imshow(img)
        plt.title(target_class)
        plt.axis('off')
        plt.show()
    
    print(f'Image shape: {img.shape}') # Show the shape of image

In [45]:
view_random_image('pizza_steak/train/', 'pizza')

PermissionError: [Errno 13] Permission denied: 'pizza_steak/train/pizza'